# Lux AI Challenge Demo
This notebook is a short demo which provides several handy functions for creating new bots, and a basic framework for the logic of a baseline bot.


## A note on types
I tried to keep my variable naming conventions as consistent as possible, both with themselves and with the game. Here are two of the most important types to keep track of, as they can often be mistaken for one another.

**Position:** any variable name with the full word "position" is a Position object, provided by the game. This will have attributes x and y for the two coordinates. On the other hand, I find it useful to use the shortened "pos" variable name if I was using a tuple in the form (x, y), though I did not use any tuple formats in this example.

**Cell:** any variable name with the word "cell" is a Cell object. This will have an attribute pos which stores a Position object. It is important to remember that a Cell IS NOT a Position, but it has a Position.


## Helper Functions
Probably the most helpful feature I have included in the demo are the helper functions. These were blocks of code that I find myself using a lot, so I stuck them into neatly packaged functions. I also attempted to maintain good naming conventions and be as clear as possible with types.

**researched(resource):** Given a Resource object, returns True if the resource type has been researched by the player.

**get_cells(cell_type):** Returns a list of Cell objects of the type specified. I created a list of preset types that tend to be useful: resource, researched resource, player citytile, enemy citytile, empty. This may be slightly different than some of the other demos which use a find_resource() function to return a list of Resource objects. I do not do this to maintain consistency with the Cell type, and because any Cell with a resource on it can return the Resource object with Cell.resource.

**find_nearest_position(target_position, option_positions):** Returns the nearest position from option_positions to target_position. Both target_position and the elements of the option_positions list should be Position objects (hence the use of the full word). However, since it is often annoying to convert a list of Cells to their corresponding Position objects, the option_positions param can also take a list of Cells or Units, which will automatically be converted to Positions.

**move_unit(unit, position):** Moves the given unit towards the given position. This function adds some useful functionality to the basic unit.move() action. First, it does a basic check for collisions, so no two Units ever attempt to move to the same cell. Next, I have it creating lines to show where the unit is attempting to travel.

**go_home(unit):** A simple function which moves the unit towards the nearest citytile.


## The Algorithm
To avoid giving away too much strategy, I have kept this algorithm very brief. It does not build new citytiles and is not optimized for faster resource collection. However, it does provide a nice example of how you might structure your code or start developing strategies. I have also marked a place to insert a conditional statement to build new citytiles. 

## Happy Coding!
Thanks for reading through everything, if you did. I hope this notebook will be of use for anyone hoping to try out the Lux AI Challenge!

In [ ]:
!pip install kaggle-environments -U

In [ ]:
from kaggle_environments import make
import json

In [ ]:
%%writefile agent.py

# housekeeping
import math, sys

if __package__ == "":
    # for kaggle-environments
    from lux.game import Game
    from lux.game_map import Cell, RESOURCE_TYPES, Position
    from lux.game_objects import Unit
    from lux.constants import Constants
    from lux.game_constants import GAME_CONSTANTS
    from lux import annotate
else:
    # for CLI tool
    from .lux.game import Game
    from .lux.game_map import Cell, RESOURCE_TYPES, Position
    from .lux.constants import Constants
    from .lux.game_constants import GAME_CONSTANTS
    from .lux import annotate

DIRECTIONS = Constants.DIRECTIONS
game_state = None


def agent(observation, configuration):
    global game_state

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    
    actions = []
    test_actions = []

    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height

    ##############################
    ### NOVEL CODE STARTS HERE ###
    ##############################

    # helper functions
    def researched(resource):
        """
        given a Resource object, return whether the player has researched the resource type
        """
        if resource.type == Constants.RESOURCE_TYPES.WOOD:
            return True
        if resource.type == Constants.RESOURCE_TYPES.COAL \
            and player.research_points >= GAME_CONSTANTS['PARAMETERS']['RESEARCH_REQUIREMENTS']['COAL']:
                return True
        if resource.type == Constants.RESOURCE_TYPES.URANIUM \
            and player.research_points >= GAME_CONSTANTS['PARAMETERS']['RESEARCH_REQUIREMENTS']['URANIUM']:
                return True
        return False

    def get_cells(cell_type):  # resource, researched resource, player citytile, enemy citytile, empty
        """
        Given a cell type, returns a list of Cell objects of the given type
        Options are: ['resource', 'researched resource', 'player citytile', 'enemy citytile', 'empty']
        """
        cells_of_type = []
        for y in range(height):
            for x in range(width):
                cell = game_state.map.get_cell(x, y)
                if (
                       ( cell_type == 'resource' and cell.has_resource() ) \
                    or ( cell_type == 'researched resource' and cell.has_resource() and researched(cell.resource) ) \
                    or ( cell_type == 'player citytile' and cell.citytile is not None and cell.citytile.team == observation.player ) \
                    or ( cell_type == 'enemy citytile' and cell.citytile is not None and cell.citytile.team != observation.player ) \
                    or ( cell_type == 'empty' and cell.citytile is None and not cell.has_resource() )
                ):
                    cells_of_type.append(cell)
        
        return cells_of_type

    def find_nearest_position(target_position, option_positions):
        """
        target_position: Position object
        option_positions: list of Position, Cell, or Unit objects (must all be the same type)
        finds the closest option_position to the target_position
        """

        # convert option_positions list to Position objects
        if type(option_positions[0]) in [Cell, Unit]:
            option_positions = [cell.pos for cell in option_positions]
            
        # find closest position
        closest_dist = math.inf
        closest_position = None
        for position in option_positions:
            dist = target_position.distance_to(position)
            if dist < closest_dist:
                closest_dist = dist
                closest_position = position

        return closest_position

    target_tiles = [] # to help avoid collisions
    def move_unit(unit, position):
        """
        moves the given unit towards the given position
        also checks basic collision detection, and adds annotations for any movement
        """

        direction = unit.pos.direction_to(position)
        target_tile = unit.pos.translate(direction, 1)
        
        # if target_tile is not being targeted already, move there
        if target_tile not in target_tiles or target_tile in [tile.pos for tile in citytile_cells]:
            target_tiles.append(target_tile)
            actions.append(unit.move(direction))
            actions.append(annotate.line(unit.pos.x, unit.pos.y, position.x, position.y))

        # else, mark an X on the map
        else:
            actions.append(annotate.x(target_tile.x, target_tile.y))

    def go_home(unit):
        """
        moves the given unit towards the nearest citytile
        """

        nearest_citytile_position = find_nearest_position(unit.pos, citytile_cells)
        move_unit(unit, nearest_citytile_position)


    #############################
    ### ALGORITHM STARTS HERE ###
    #############################

    # get all resource tiles
    researched_resource_cells = get_cells('researched resource')
    citytile_cells = get_cells('player citytile')

    # calculate number of citytiles
    num_citytiles = len(citytile_cells)

    # iterate over units
    for unit in player.units:
        if unit.is_worker() and unit.can_act():

            # if night and there are cities, return home:
            if game_state.turn % 40 > 30 and len(player.cities) > 0:
                go_home(unit)

            # if there is cargo space, find nearest resource and move towards it
            elif unit.get_cargo_space_left() > 0:
                nearest_resource_position = find_nearest_position(unit.pos, researched_resource_cells)
                move_unit(unit, nearest_resource_position)

            # if cargo is full
            else:
                # if there are no cities, build one if possible
                if len(player.cities) == 0:
                    if unit.can_build(game_state.map):
                        actions.append(unit.build_city)

                elif False: # some build condition here
                    if unit.can_build(game_state.map):
                        actions.append(unit.build_city)

                else:
                    nearest_citytile_position = find_nearest_position(unit.pos, citytile_cells)
                    move_unit(unit, nearest_citytile_position)
                
   # iterate through cities
    for k, city in player.cities.items():
        for citytile in city.citytiles:
            if citytile.can_act():

                # if there is space for more units, build a worker
                if num_citytiles > len(player.units):
                    actions.append(citytile.build_worker())
                
                # else research
                else:
                    actions.append(citytile.research())

    return actions


In [ ]:
env = make('lux_ai_2021', configuration={'seed': 562124210, 'loglevel': 2, 'annotations': True}, debug=True)
steps = env.run(['agent.py', 'simple_agent'])

In [ ]:
env.render(mode='ipython', width=1200, height=1000)